In [14]:
processed_dataset = processed_dataset.rename_column("label_id", "labels")
display(processed_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1703
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 426
    })
})

In [11]:
display(processed_dataset)
display(processed_dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['reply', 'label', 'label_id', 'input_ids', 'attention_mask'],
        num_rows: 1703
    })
    test: Dataset({
        features: ['reply', 'label', 'label_id', 'input_ids', 'attention_mask'],
        num_rows: 426
    })
})

{'reply': 'Not good fit for team',
 'label': 'Negative',
 'label_id': 8,
 'input_ids': [101,
  2025,
  2204,
  4906,
  2005,
  2136,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [17]:
trainer.save_model("./fine_tuned_model")

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    processing_class=tokenizer, # Use processing_class instead of tokenizer
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.561654,0.821596,0.819889
2,No log,0.483740,0.826291,0.822964
3,No log,0.457247,0.823944,0.821693


TrainOutput(global_step=321, training_loss=0.7015019592095015, metrics={'train_runtime': 256.0169, 'train_samples_per_second': 19.956, 'train_steps_per_second': 1.254, 'total_flos': 676860425100288.0, 'train_loss': 0.7015019592095015, 'epoch': 3.0})

In [16]:
eval_results = trainer.evaluate()
display(eval_results)

{'eval_loss': 0.4572474956512451,
 'eval_accuracy': 0.823943661971831,
 'eval_f1': 0.8216925103344043,
 'eval_runtime': 6.2915,
 'eval_samples_per_second': 67.711,
 'eval_steps_per_second': 4.292,
 'epoch': 3.0}

# Task
Fine-tune a small transformer model (e.g., distilbert-base-uncased) using Hugging Face for a text classification task. The dataset is located at "/content/reply_classification_dataset.csv". Evaluate the model using accuracy and F1 score.

## Install necessary libraries

### Subtask:
Install the `transformers` and `datasets` libraries from Hugging Face, as well as `accelerate` and `scikit-learn` for training and evaluation.


**Reasoning**:
Install the necessary libraries for the task.



In [1]:
%pip install transformers datasets accelerate scikit-learn

## Load a dataset

### Subtask:
Load the dataset from "/content/reply_classification_dataset.csv" into a pandas DataFrame.


**Reasoning**:
Load the dataset into a pandas DataFrame and display its head and info to understand its structure and content.



In [2]:
import pandas as pd

df = pd.read_csv("/content/reply_classification_dataset.csv")
display(df.head())
display(df.info())

,reply,label
0,Can we discuss pricing??,NEUTRAL
1,"Im excited to explore this further, plz send c...",POSITIVE
2,We not looking for new solutions.,negative
3,Could u clarify features included?,neutral
4,"lets,, schedule a meeting to dive deeper",positive


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   reply   2129 non-null   object
 1   label   2129 non-null   object
dtypes: object(2)
memory usage: 33.4+ KB


None

## Preprocess the data

### Subtask:
Tokenize the text data and format it for input to the transformer model. Convert the pandas DataFrame to a Hugging Face dataset and split it into training and validation sets.


**Reasoning**:
Map labels to numerical IDs, load the tokenizer, define the tokenization function, convert to Hugging Face Dataset, apply tokenization, and split the dataset.



In [12]:
from datasets import Dataset
from transformers import AutoTokenizer

# 1. Map labels to numerical IDs
unique_labels = df['label'].unique()
label_to_id = {label: i for i, label in enumerate(unique_labels)}
df['label_id'] = df['label'].map(label_to_id)

# 2. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 3. Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["reply"], padding="max_length", truncation=True)

# 4. Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# 5. Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 6. Split the dataset and remove original columns
processed_dataset = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
processed_dataset = processed_dataset.remove_columns(["reply", "label"]) # Remove original columns

display(processed_dataset)

Map:   0%|          | 0/2129 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label_id', 'input_ids', 'attention_mask'],
        num_rows: 1703
    })
    test: Dataset({
        features: ['label_id', 'input_ids', 'attention_mask'],
        num_rows: 426
    })
})

## Load the pre-trained model

### Subtask:
Load the `distilbert-base-uncased` model and a suitable classification head from the Hugging Face transformers library.


**Reasoning**:
Import the necessary class from the transformers library, determine the number of unique labels from the existing `label_to_id` variable, and load the specified pre-trained model with the correct number of output labels for the classification head.



In [4]:
from transformers import AutoModelForSequenceClassification

num_labels = len(label_to_id)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

display(model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Define evaluation metrics

### Subtask:
Define functions to compute accuracy and F1 score for evaluating the model's performance.


**Reasoning**:
Define the compute_metrics function using accuracy_score and f1_score from sklearn.metrics as per the instructions.



In [5]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}

## Configure training arguments

### Subtask:
Set up the training parameters, such as the number of epochs, learning rate, and batch size.


**Reasoning**:
Import the `TrainingArguments` class and instantiate it with the specified parameters to set up the training configuration.



In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

display(training_args)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

**Reasoning**:
The previous attempt failed because `evaluation_strategy` is not a valid argument for `TrainingArguments` in the current version. The correct argument is `eval_strategy`. Correct the argument name and instantiate `TrainingArguments` again.



In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Corrected argument name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none", # Disable Weights & Biases logging
)

display(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

## Train the model

### Subtask:
Train the model using the `Trainer` class from the Hugging Face transformers library.


**Reasoning**:
Import the `Trainer` class, instantiate it with the model, training arguments, datasets, tokenizer, and compute_metrics function, and then start the training.



In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

# Now you can use the loaded model for inference

In [19]:
display(model.config)

DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "dtype": "float32",
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.56.1",
  "vocab_size": 30522
}

In [20]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer
import torch

# Load your fine-tuned model
model_path = "/path/to/your/fine-tuned-model"  # adjust path
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Update id2label mapping
model.config.id2label = {0: "negative", 1: "neutral", 2: "positive"}
model.config.label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/path/to/your/fine-tuned-model'. Use `repo_type` argument if needed.

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

# Define a sample text for prediction
sample_text = "This is a great product!"

# Tokenize the input text
inputs = tokenizer(sample_text, return_tensors="pt")

# Make a prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Get the predicted label
predicted_label_id = predictions.item()

# Map the predicted label ID back to the original label string
# You can use the id2label mapping from the model's config
predicted_label = model.config.id2label[predicted_label_id]

print(f"Sample Text: {sample_text}")
print(f"Predicted Label ID: {predicted_label_id}")
print(f"Predicted Label: {predicted_label}")

Sample Text: This is a great product!
Predicted Label ID: 1
Predicted Label: LABEL_1
